In [ ]:
import numpy as np
from scipy.integrate import odeint

def lotka_volterra_model(y: np.ndarray, t: float,
        alpha: float, beta: float, gamma: float, delta: float) -> np.ndarray:
    u, v = y
    return np.array([(alpha - beta * v) * u, (-gamma + delta * u) * v])

def ode(y: np.ndarray, t: float,
        alpha: float, beta: float, gamma: float, delta: float) -> np.ndarray:
    (u, v, u_alpha, u_beta, u_gamma, u_delta, v_alpha, v_beta, v_gamma, v_delta
    ) = y
    return np.array([
        (alpha - beta * v) * u,
        (-gamma + delta * u) * v,

        u + (alpha - beta * v) * u_alpha + (-beta * u) * v_alpha,
        -v * u + (alpha - beta * v) * u_beta + (-beta * u) * v_beta,
        (alpha - beta * v) * u_gamma + (-beta * u) * v_gamma,
        (alpha - beta * v) * u_delta + (-beta * u) * v_delta,

        delta * v * u_alpha + (-gamma + delta * u) * v_alpha,
        delta * v * u_beta + (-gamma + delta * u) * v_beta,
        -v + delta * v * u_gamma + (-gamma + delta * u) * v_gamma,
        u * v + delta * v * u_delta + (-gamma + delta * u) * v_delta
    ])

def func(x: np.ndarray, *args: tuple) -> float:
    t, u, v = args
    y = odeint(lotka_volterra_model, np.array([u[0], v[0]]), t, args=tuple(x))
    return np.sum(np.power(y[:, 0] - u, 2)) + np.sum(np.power(y[:, 1] - v, 2))

def func_jac(x: np.ndarray, *args: tuple) -> np.ndarray:
    n = len(x)
    t, u, v = args
    y0 = np.zeros(2 + 2 * n)
    y0[0], y0[1] = u[0], v[0]
    y = odeint(ode, y0=y0, t=t, args=tuple(x))
    result = np.zeros(n)
    for i in range(n):
        result[i] += np.sum((y[:, 0] - u) * y[:, 2 + i])
        result[i] += np.sum((y[:, 1] - v) * y[:, 2 + n + i])
    result *= 2.0
    return result

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import minimize
np.random.seed(4167839052)

In [ ]:
start, stop, n = 0.0, 20.0, 200
t = np.linspace(start, stop, n)

alpha, beta, gamma, delta = 0.42, 0.14, 0.79, 0.17
y0 = np.array([15.0, 10.0])

sol = odeint(lotka_volterra_model, y0, t, args=(alpha, beta, gamma, delta))
u_e = sol[:, 0]
v_e = sol[:, 1]

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white', title='predator-prey')
subplot.plot(t, sol[:, 1], 'red', label='predator', linewidth=2)
subplot.plot(t, sol[:, 0], 'green', label='prey', linewidth=2)

plt.legend()
plt.show()

In [ ]:
plt.plot(sol[:, 0], sol[:, 1])
prcnt = 0.05
u_e += prcnt * sol[:, 0] * np.random.randn(n)
v_e += prcnt * sol[:, 1] * np.random.randn(n)

In [ ]:
for i in range(10):
    rand_x0 = np.random.rand(4)
    print('x0 = ', rand_x0)
    res = minimize(fun=func, x0=rand_x0, args=(t, u_e, v_e), method='Nelder-Mead', tol=1e-3)
    print('x = ', res.x)
    print('err = ', res.fun)
    print('nit = ', res.nit)
    print()

In [ ]:
for i in range(10):
    rand_x0 = np.random.rand(4)
    print('x0 = ', rand_x0)
    res = minimize(fun=func, x0=rand_x0, args=(t, u_e, v_e), method='TNC', jac=func_jac)
    print('x = ', res.x)
    print('err = ', res.fun)
    print('nit = ', res.nit)
    print()

In [ ]:
args = (0.41572561, 0.13543324, 0.76847345, 0.16055372)
tn = np.linspace(0, 80, 400)
sol_new = odeint(lotka_volterra_model, y0, t, args=args)

plt.plot(sol[:, 0], sol[:, 1])
plt.plot(sol_new[:, 0], sol_new[:, 1])